# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [2]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [61]:
df = data.groupby(['CustomerID', 'ProductName'], as_index=False).agg({'Quantity':'sum'})
df

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [64]:
pivot = pd.pivot_table(df,
                      index = 'ProductName',
                    columns = 'CustomerID',
                       values= 'Quantity',
                      aggfunc = 'sum').fillna(0)
pivot

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
"Yogurt - Blueberry, 175 Gr",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [169]:
matrix_similarity = pd.DataFrame(1/(1+squareform(pdist(pivot.T, metric = "euclidean"))), index = pivot.columns, columns = pivot.columns)
matrix_similarity
matrix_similarity_2 = pd.DataFrame(1/(1+squareform(pdist(pivot.T, metric = "canberra"))), index = pivot.columns, columns = pivot.columns)
matrix_similarity_2

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.008596,0.010563,0.010067,0.009009,0.009554,0.008596,0.009375,0.009202,0.009554,...,0.008238,0.008649,0.008667,0.009028,0.008521,0.008425,0.008376,0.008500,0.007661,0.008447
200,0.008596,1.000000,0.008671,0.008571,0.007472,0.008130,0.008496,0.008264,0.008671,0.008955,...,0.007813,0.008042,0.008209,0.008606,0.007798,0.007784,0.008215,0.007980,0.007071,0.007557
264,0.010563,0.008671,1.000000,0.009231,0.009494,0.009009,0.008608,0.008850,0.009009,0.008982,...,0.008398,0.008587,0.008442,0.009226,0.008607,0.009018,0.008547,0.008675,0.007943,0.008238
356,0.010067,0.008571,0.009231,1.000000,0.008772,0.008902,0.008671,0.009967,0.008772,0.009091,...,0.008248,0.008581,0.008595,0.009213,0.008374,0.008597,0.008022,0.008751,0.007934,0.008384
412,0.009009,0.007472,0.009494,0.008772,1.000000,0.008646,0.008621,0.009464,0.008499,0.009091,...,0.007669,0.008549,0.008037,0.008742,0.008051,0.008246,0.007720,0.008183,0.007636,0.008349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.008425,0.007784,0.009018,0.008597,0.008246,0.008681,0.008263,0.008904,0.008603,0.008592,...,0.008696,0.009709,0.008772,0.010169,0.009631,1.000000,0.009615,0.009677,0.007958,0.008086
98069,0.008376,0.008215,0.008547,0.008022,0.007720,0.008095,0.007857,0.008442,0.008312,0.008452,...,0.008264,0.008798,0.008596,0.008929,0.009375,0.009615,1.000000,0.009494,0.008086,0.008850
98159,0.008500,0.007980,0.008675,0.008751,0.008183,0.008595,0.008183,0.008737,0.008763,0.008667,...,0.008621,0.009868,0.009231,0.009434,0.009009,0.009677,0.009494,1.000000,0.008671,0.008646


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [69]:
matrix_similarity[33].sort_values(ascending=False)[1:6]

CustomerID
3317    0.087047
3535    0.087047
264     0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [89]:
customers = df.loc[df.loc[:,'CustomerID'].isin([3317,3535,264,2503,3305])]
customers

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1
...,...,...,...
3003,3535,Tray - 16in Rnd Blk,1
3004,3535,Wanton Wrap,1
3005,3535,Wine - Blue Nun Qualitatswein,1
3006,3535,"Wine - Magnotta, Merlot Sr Vqa",1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [96]:
ranking = customers.groupby('ProductName').agg({'Quantity':'sum'}).reset_index().sort_values(by = 'Quantity', ascending=False)
ranking

,ProductName,Quantity
33,Butter - Unsalted,3
203,Wine - Ej Gallo Sierra Valley,3
186,Towels - Paper / Kraft,3
168,Soup - Campbells Bean Medley,3
199,Wine - Blue Nun Qualitatswein,3
...,...,...
85,Hinge W Undercut,1
86,Ice Cream Bar - Hageen Daz To,1
87,Jagermeister,1
88,Jolt Cola - Electric Blue,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [136]:
general = ranking.merge(pivot, on = 'ProductName')
not_purchased = general[general[33]==0]['ProductName'].reset_index(drop=True)
not_purchased

0                  Butter - Unsalted
1      Wine - Ej Gallo Sierra Valley
2       Soup - Campbells Bean Medley
3      Wine - Blue Nun Qualitatswein
4                Chicken - Soup Base
                   ...              
182                 Halibut - Steaks
183                 Hinge W Undercut
184    Ice Cream Bar - Hageen Daz To
185                     Jagermeister
186        Jolt Cola - Electric Blue
Name: ProductName, Length: 187, dtype: object

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [137]:
recommendations = {}
customersId = list(data['CustomerID'].unique())

for customer in customersId:
    similars = matrix_similarity[customer].sort_values(ascending=False)[1:6].index
    for_customers = df.loc[df.loc[:,'CustomerID'].isin(similars)]
    ranking_for = for_customers.groupby('ProductName').agg({'Quantity':'sum'}).reset_index().sort_values(by = 'Quantity', ascending=False)
    general_for = ranking_for.merge(pivot, on = 'ProductName')
    for similar in similars:
        for_not_purchased = general_for[general_for[similar]==0]['ProductName'][:5].reset_index(drop=True)
        recommendations[customer] = for_not_purchased
        


In [138]:
recommendations

{61288: 0               Beef - Rib Eye Aaa
 1              Chicken - Soup Base
 2             Lamb - Pieces, Diced
 3              Flavouring - Orange
 4    Wine - Prosecco Valdobiaddene
 Name: ProductName, dtype: object,
 77352: 0         Tuna - Salad Premix
 1         Chicken - Soup Base
 2    Soup - Campbells, Lentil
 3                     Isomalt
 4           Wiberg Super Cure
 Name: ProductName, dtype: object,
 40094: 0    Wine - White, Colubia Cresh
 1           Cheese - Brie,danish
 2         Cookie - Dough Variety
 3                   Sherry - Dry
 4                 Shrimp - 31/40
 Name: ProductName, dtype: object,
 23548: 0                Spinach - Baby
 1     Jolt Cola - Electric Blue
 2           Flavouring - Orange
 3    Bread - Raisin Walnut Oval
 4                Banana Turning
 Name: ProductName, dtype: object,
 78981: 0           Lettuce - Frisee
 1      Pop Shoppe Cream Soda
 2        Beef - Inside Round
 3    Creme De Banane - Marie
 4       Liners - Baking Cups
 Name

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [166]:
df_recommendations = pd.DataFrame.from_dict(recommendations, orient = 'index').reset_index()
df_recommendations

,index,0,1,2,3,4
0,61288,Beef - Rib Eye Aaa,Chicken - Soup Base,"Lamb - Pieces, Diced",Flavouring - Orange,Wine - Prosecco Valdobiaddene
1,77352,Tuna - Salad Premix,Chicken - Soup Base,"Soup - Campbells, Lentil",Isomalt,Wiberg Super Cure
2,40094,"Wine - White, Colubia Cresh","Cheese - Brie,danish",Cookie - Dough Variety,Sherry - Dry,Shrimp - 31/40
3,23548,Spinach - Baby,Jolt Cola - Electric Blue,Flavouring - Orange,Bread - Raisin Walnut Oval,Banana Turning
4,78981,Lettuce - Frisee,Pop Shoppe Cream Soda,Beef - Inside Round,Creme De Banane - Marie,Liners - Baking Cups
...,...,...,...,...,...,...
995,49005,"Pasta - Penne, Rigate, Dry",Lettuce - Spring Mix,Browning Caramel Glace,Ecolab - Mikroklene 4/4 L,"Garlic - Primerba, Paste"
996,41286,Beef - Ground Medium,Veal - Inside,Wonton Wrappers,Bread Crumbs - Japanese Style,Pecan Raisin - Tarts
997,85878,Olives - Kalamata,Wine - White Cab Sauv.on,Bread - Calabrese Baguette,Cheese - Wine,Wine - Two Oceans Cabernet
998,68506,Onions - Cippolini,"Veal - Brisket, Provimi,bnls",Beef - Ground Medium,Scampi Tail,"Rosemary - Primerba, Paste"


In [168]:
df_recommendations.columns= ['CustomerID', '1st Recommendation','2nd Recommendation', '3rd Recommendation','4th Recommendation', '5th Recommendation']
df_recommendations

,CustomerID,1st Recommendation,2nd Recommendation,3rd Recommendation,4th Recommendation,5th Recommendation
0,61288,Beef - Rib Eye Aaa,Chicken - Soup Base,"Lamb - Pieces, Diced",Flavouring - Orange,Wine - Prosecco Valdobiaddene
1,77352,Tuna - Salad Premix,Chicken - Soup Base,"Soup - Campbells, Lentil",Isomalt,Wiberg Super Cure
2,40094,"Wine - White, Colubia Cresh","Cheese - Brie,danish",Cookie - Dough Variety,Sherry - Dry,Shrimp - 31/40
3,23548,Spinach - Baby,Jolt Cola - Electric Blue,Flavouring - Orange,Bread - Raisin Walnut Oval,Banana Turning
4,78981,Lettuce - Frisee,Pop Shoppe Cream Soda,Beef - Inside Round,Creme De Banane - Marie,Liners - Baking Cups
...,...,...,...,...,...,...
995,49005,"Pasta - Penne, Rigate, Dry",Lettuce - Spring Mix,Browning Caramel Glace,Ecolab - Mikroklene 4/4 L,"Garlic - Primerba, Paste"
996,41286,Beef - Ground Medium,Veal - Inside,Wonton Wrappers,Bread Crumbs - Japanese Style,Pecan Raisin - Tarts
997,85878,Olives - Kalamata,Wine - White Cab Sauv.on,Bread - Calabrese Baguette,Cheese - Wine,Wine - Two Oceans Cabernet
998,68506,Onions - Cippolini,"Veal - Brisket, Provimi,bnls",Beef - Ground Medium,Scampi Tail,"Rosemary - Primerba, Paste"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [170]:
recommendations = {}
customersId = list(data['CustomerID'].unique())

for customer in customersId:
    similars = matrix_similarity_2[customer].sort_values(ascending=False)[1:6].index
    for_customers = df.loc[df.loc[:,'CustomerID'].isin(similars)]
    ranking_for = for_customers.groupby('ProductName').agg({'Quantity':'sum'}).reset_index().sort_values(by = 'Quantity', ascending=False)
    general_for = ranking_for.merge(pivot, on = 'ProductName')
    for similar in similars:
        for_not_purchased = general_for[general_for[similar]==0]['ProductName'][:5].reset_index(drop=True)
        recommendations[customer] = for_not_purchased

In [172]:
df_recommendations = pd.DataFrame.from_dict(recommendations, orient = 'index').reset_index()
df_recommendations.columns= ['CustomerID', '1st Recommendation','2nd Recommendation', '3rd Recommendation','4th Recommendation', '5th Recommendation']
df_recommendations

,CustomerID,1st Recommendation,2nd Recommendation,3rd Recommendation,4th Recommendation,5th Recommendation
0,61288,Bandage - Fexible 1x3,"Oil - Shortening,liqud, Fry",Wine - Toasted Head,Mangoes,"Cheese - Brie, Triple Creme"
1,77352,Grenadine,"Wine - Red, Colio Cabernet","Pork - Loin, Bone - In",Halibut - Fletches,Wine - White Cab Sauv.on
2,40094,Muffin - Zero Transfat,Sponge Cake Mix - Chocolate,Water - Aquafina Vitamin,Jolt Cola - Electric Blue,Wine - Hardys Bankside Shiraz
3,23548,"Lamb - Pieces, Diced",Cheese - Parmesan Cubes,Wine - Ruffino Chianti,Muffin - Zero Transfat,Mussels - Frozen
4,78981,Duck - Breast,Muffin - Zero Transfat,Cheese - Parmesan Cubes,Water - Aquafina Vitamin,Bread - Italian Roll With Herbs
...,...,...,...,...,...,...
995,49005,Soupcontfoam16oz 116con,Dried Figs,Beef - Striploin Aa,"Turkey - Whole, Fresh","Soup - Campbells, Cream Of"
996,41286,French Pastry - Mini Chocolate,"Lemonade - Natural, 591 Ml",Jagermeister,Pail For Lid 1537,"Juice - Cranberry, 341 Ml"
997,85878,Mussels - Frozen,Bread - French Baquette,Beer - Labatt Blue,"Beef - Tenderlion, Center Cut",Wine - Sogrape Mateus Rose
998,68506,Chocolate - Dark,Lettuce - Frisee,Muffin Mix - Blueberry,"Beef - Ground, Extra Lean, Fresh",Tuna - Salad Premix
